In [1]:
from settings import s, e
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import pickle

from IPython.display import clear_output, display, update_display
import matplotlib.pyplot as plt


from simple import Game, get_x

from agent_code.tensor_agent.agent import TensorAgent
from agent_code.tensor_agent.hyperparameters import hp
from agent_code.tensor_agent.X import RelativeX2 as game_state_X

clear_output()

In [2]:
choices = ['RIGHT', 'LEFT', 'UP', 'DOWN', 'BOMB', 'WAIT']
D = len(choices)

K.clear_session()
agents = {
    'tensor_agent': TensorAgent(game_state_X.shape, D, weights=None) # alt: weights='tensor_agent-model.h5'
}
train = {a: False for a in agents}
clear_output()

In [3]:
d = display(f'Starting...', display_id='progress')

n_steps = 1000
game = None

train['tensor_agent'] = True

for step in range(n_steps):
    if game is None:
        game = Game(*Game.create_arena(agents.keys()))

    actions = {}
    Xs = {}
    for agent in game.agents:
        _, _, name, _, _ = agent
        
        game_state = game.get_game_state(agent)
        Xs[name] = game_state_X.get(game_state)
        actions[name] = agents[name].act(Xs[name], train=train[name])
        
    game.step(actions)
    
    rewards = {'tensor_agent': 1.}

    for _, _, name, _, _ in game.agents:
        if train[name]:
            agents[name].reward_update([Xs[name], actions[name], rewards[name]])
    
    if game.terminated:
        for name, a in agents.items():
            a.end_of_episode(save='tensor_agent-model.h5') # alt: save=None
        
        d.update(f'Step: {step+1}/{n_steps}')
        
        game = None

d.update(f'Complete')

'Complete'

End of episode. Steps: 400
End of episode. Steps: 800
